In [1]:
%pylab
%matplotlib

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
Using matplotlib backend: MacOSX


In [2]:
import nibabel as nib
import numpy as np
from dipy.tracking import utils
from dipy.tracking.eudx import EuDX
from dipy.reconst import peaks, shm

from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table

import matplotlib.pyplot as plt

from dipy.segment.clustering import QuickBundles
from dipy.viz import fvtk

paths

In [3]:
dname = '/Volumes/Samsung_T1/dti/Dipy/P2_F25/'
ftrk = dname + 'filteredtractogram.trk'
ffa = dname + 'tensor_fa.nii.gz'
fdwi = dname + '2dseq.src.gz.nii.gz'

#fbval = dname + 'bvals'
#fbvec = dname + 'bvecs'

#fvtk = dname + 'adult-F01.vtk'
show = False

load FA and DWI data

In [4]:
fa_img = nib.load(ffa)
fa = fa_img.get_data()
affine = fa_img.get_affine()

img = nib.load(fdwi)
data = img.get_data()

load streamlines and render if show=True

In [5]:
from nibabel import trackvis
streams, hdr = trackvis.read(ftrk)
streamlines = [s[0] for s in streams]

#from dipy.tracking.streamline import transform_streamlines
#streamlines2 = transform_streamlines(streamlines,affine)

if show:
    from dipy.viz import actor, window
    ren = window.Renderer()
    ren.add(actor.line(streamlines[:1000]))
    window.show(ren)


compute density map

In [6]:
dm = utils.density_map(streamlines, fa.shape, affine=affine)

save density map in nifti

In [7]:
dm_img = nib.Nifti1Image(dm.astype("int16"), affine)
dm_img.to_filename("fa-dm.nii.gz")

save pictures of the density map

In [9]:
dm.shape

(60, 128, 40)

In [10]:
plt.figure(1)
plt.pcolor(dm[:,:,20])
plt.colorbar()
plt.savefig("densitymap_z.png")
plt.figure(2)
plt.pcolor(dm[:,64,:])
plt.colorbar()
plt.savefig("densitymap_y.png")
plt.figure(3)
plt.pcolor(dm[30,:,:])
plt.colorbar()
plt.savefig("densitymap_x.png")

load density matrix from nii

In [11]:
a = nib.load("fa-dm.nii.gz")
da = a.get_data()

plot distribution of the density map of one slice

In [13]:
fig_hist, ax = plt.subplots(1)
ax.hist(dm[:,64,20])
ax.set_xlabel('Voxel')
ax.set_ylabel('Count')
plt.show()
plt.legend()

/Users/ghfc/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


clustering similar streamlines / QuickBundles

In [14]:
qb = QuickBundles(threshold=1.)
clusters = qb.cluster(streamlines)

In [15]:
print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

('Nb. clusters:', 348)
('Cluster sizes:', [56, 195, 106, 38, 79, 50, 96, 181, 14, 42, 68, 31, 53, 2, 211, 28, 95, 124, 47, 87, 122, 78, 116, 14, 5, 73, 15, 100, 44, 50, 39, 44, 151, 4, 39, 14, 25, 21, 29, 55, 63, 17, 18, 18, 47, 12, 15, 14, 5, 12, 19, 64, 15, 2, 3, 62, 33, 5, 117, 35, 3, 6, 17, 3, 80, 18, 6, 41, 28, 24, 67, 1, 23, 22, 56, 41, 43, 1, 43, 83, 73, 117, 104, 66, 1, 15, 10, 69, 22, 74, 14, 58, 105, 17, 30, 1, 1, 1, 2, 3, 2, 37, 11, 5, 127, 14, 1, 29, 1, 8, 24, 97, 1, 5, 3, 2, 1, 5, 18, 15, 30, 137, 1, 17, 14, 12, 33, 83, 12, 2, 9, 100, 1, 138, 7, 11, 9, 5, 1, 58, 49, 2, 6, 16, 40, 32, 2, 12, 20, 76, 42, 82, 10, 28, 39, 37, 1, 17, 28, 29, 12, 2, 7, 24, 2, 6, 3, 13, 13, 43, 1, 8, 26, 19, 15, 57, 26, 7, 19, 22, 26, 1, 73, 121, 17, 7, 5, 16, 104, 75, 23, 29, 6, 2, 78, 42, 25, 103, 5, 159, 35, 81, 18, 115, 68, 23, 32, 22, 128, 28, 52, 13, 4, 39, 44, 12, 14, 118, 27, 3, 16, 113, 39, 12, 34, 26, 3, 24, 62, 13, 33, 22, 120, 1, 65, 46, 2, 129, 5, 14, 7, 1, 22, 55, 18, 52, 31, 2, 11,

render bundles and centroids (& save images)

In [16]:
ren = fvtk.ren()
ren.SetBackground(1, 1, 1)

In [17]:
fvtk.add(ren, fvtk.streamtube(streamlines, fvtk.colors.white))
fvtk.record(ren, n_frames=1, out_path=dname+'fornix_initial.png', size=(600, 600))

In [18]:
colormap = fvtk.create_colormap(np.arange(len(clusters)))

colormap_full = np.ones((len(streamlines), 3))
for cluster in clusters:
    colormap_full[cluster.indices] = np.random.rand(3)

In [19]:
fvtk.clear(ren)
ren.SetBackground(1, 1, 1)
#fvtk.add(ren, fvtk.streamtube(streamlines, fvtk.colors.white, opacity=0.05))
fvtk.add(ren, fvtk.line(clusters.centroids, linewidth=0.4, opacity=1))
#fvtk.record(ren, n_frames=1, out_path='fornix_centroids.png', size=(600, 600))

In [20]:
fvtk.clear(ren)
ren.SetBackground(1, 1, 1)
fvtk.add(ren, fvtk.line(streamlines, colormap_full))
fvtk.record(ren, n_frames=1, out_path='fornix_clusters.png', size=(600, 600))

In [21]:
fvtk.show(ren)

In [22]:
fvtk.clear(ren)

Distance (between centroids) matrix

In [23]:
from dipy.tracking.distances import bundles_distances_mdf

D = bundles_distances_mdf(clusters.centroids, clusters.centroids)

In [24]:
imshow(D)